# 네이버 항공권 웹스크래핑

In [113]:
#### Q. 네이버 항공권
# 김포, 제주 5/31 ~ 6/1 ->
# 출발 비행기표 금액 110,000원 이상 제외
# 김포출발시간 5/31 17:00 이후 제외

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv
import time
import random
from datetime import datetime

# 크롬 옵션 설정 - 셀레니움 접근 제한 : 보안접근 해제
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 티 안 나게
options.add_argument("start-maximized")  # 브라우저 창 최대화
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")  # 사용자 에이전트 변경
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [114]:
# 브라우저 실행
browser = webdriver.Chrome(options=options)
browser.maximize_window() # 창 최대화

In [115]:
### 1. 네이버 항공권 접속
url = "https://flight.naver.com/"
browser.get(url)

In [116]:
time.sleep(2)
# 팝업창 닫기
elem = browser.find_element(By.CLASS_NAME,'FullscreenPopup_close')
elem.click()

In [117]:
# 3. 출발지 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[1]/b')
elem.click()
time.sleep(2)

# 4. 김포선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div/div/ul[1]/li[3]/button')
elem.click()
time.sleep(2)

# 5. 도착지 선택
elem = browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[2]/b')
elem.click()
time.sleep(2)

# 6. 제주선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[2]/div[2]/ul[1]/li[1]/button').click()
time.sleep(2)

In [118]:
# 7. 가는날 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[2]/button[1]').click()
time.sleep(2)

# 8. 출발날짜 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div/div/div[3]/table/tbody/tr[1]/td[7]/button/b').click()
time.sleep(2)
# 9. 도착날짜 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div/div/div[3]/table/tbody/tr[2]/td[1]/button/b').click()
time.sleep(2)

# 10. 항공권 검색 선택
browser.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/button').click()

In [119]:
# 11. 항공권 출력할때까지 대기
# 검색버튼 클릭후 화면이 나타날때까지 대기하고 길게는 10초 대기
# WebDriverWait(browser,10).until(EC.presence_of_all_elements_located(By.XPATH,'//*[@id="__next"]/div/main/div[4]/div/div[2]/div[2]/div[1]'))
time.sleep(7)

### 스크롤 내리기
# 현재 사이트 높이를 가져오는 자바스크립트
prev_height = browser.execute_script("return document.body.scrollHeight")

# 반복 실행
while True:
    # 현재 브라우저에서 0에서 현재 높이까지 스크롤 내리기
    # 자바스크립트 실행
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    # 변동된 현재 문서 높이을 가져오기
    curr_height = browser.execute_script("return document.body.scrollHeight")
    # 스크롤 높이가 변동이 없을시
    if curr_height == prev_height: break 
    prev_height = curr_height # 현재 높이를 다시 입력해서 스크롤 내리기 재실행

print("스크롤 내리기 종료")

스크롤 내리기 종료


In [122]:
soup = BeautifulSoup(browser.page_source,"lxml")

## 비행정보 모두 가져오기
datas = soup.find_all("div",{"class":"domestic_Flight__8bR_b"})
print(len(datas)) #287
datas_list = []   # List정렬
for data in datas:
    ## 출발시간   "17:00" > "06:15"
    times = data.find_all("b",{"class":"route_time__xWu7a"})
    startTime = times[0].get_text().strip() ## 06:15
    
    # 기준시간
    standard_time = datetime(2025,5,31,17,00,00)

    # 검색된 시간 - "06:15"
    d_time = startTime.split(":")
    search_time = datetime(2025,5,31,int(d_time[0]),int(d_time[1]),00)

    ######    
    ## 가격 - 천단위제거
    price = data.find("i",{"class":"domestic_num__ShOub"}).get_text().strip().replace(",","")
    price = int(price)
    price
    #### 조건 140000보다 크거나, 시간이 17:00보다 크면 제외대상
    if price > 140000 or (standard_time<search_time):
        continue
    
    ## 항공사이름
    airline = data.find("b",{"class":"airline_name__0Tw5w"}).get_text().strip()
    print(airline)
    times = data.find_all("b",{"class":"route_time__xWu7a"})
    ## 출발시간
    startTime = times[0].get_text().strip()
    print(startTime)
    ## 도착시간
    endTime = times[1].get_text().strip()
    print(endTime)
    ## 가격 - 천단위제거
    price = data.find("i",{"class":"domestic_num__ShOub"}).get_text().strip().replace(",","")
    price = int(price)
    print(price)
    print('-'*50)
    # datas_list.append( [airline,startTime,endTime,price] )

# ### 최저가 LIST정렬
# dd_list = sorted(datas_list,key = lambda x:x[3]) # 순차정렬
# # dd_list = sorted(d_list,key = lambda x:x[3], reverse=True ) # 역순정렬
# print(dd_list) 

301
아시아나항공
06:00
07:10
71700
--------------------------------------------------
에어서울
06:00
07:15
76700
--------------------------------------------------
에어서울
06:00
07:15
120800
--------------------------------------------------
아시아나항공
06:00
07:10
131700
--------------------------------------------------
이스타항공
06:05
07:20
68600
--------------------------------------------------
이스타항공
06:05
07:20
118600
--------------------------------------------------
이스타항공
06:10
07:25
68600
--------------------------------------------------
제주항공
06:10
07:30
118200
--------------------------------------------------
이스타항공
06:10
07:25
118600
--------------------------------------------------
진에어
06:15
07:30
65600
--------------------------------------------------
제주항공
06:15
07:30
65600
--------------------------------------------------
대한항공
06:15
07:30
87700
--------------------------------------------------
대한항공
06:15
07:30
106700
--------------------------------------------------
제주항공
06:15
07:30
1182

In [121]:
# # import datetime
# from datetime import datetime

# # 기준시간
# standard_time = datetime(2025,5,31,17,00,00)

# # 검색된 시간 - 06:15
# times = "06:15"
# d_time = times.split(":")
# search_time = datetime(2025,5,31,int(d_time[0]),int(d_time[1]),00)

# if (standard_time < search_time):  # 기준시간보다 검색된 시간이 더 크면 제외
#     print("제외 대상입니다.")
# else:
#     print("데이터 출력대상입니다.")